<a href="https://colab.research.google.com/github/gerfranc/scraper_nosis/blob/main/Scrap_NOSIS_vf18_05_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 94.4 MB/s eta 0:00:00


In [ ]:

"""
Script optimizado para extraer información estructurada de PDFs y convertirla en DataFrames.
Versión mejorada con correcciones para manejar posiciones X y extracción de datos.
"""

# 1. IMPORTACIONES - Organizadas por tipo
import re
import os
import logging
import warnings
from concurrent.futures import ThreadPoolExecutor
from typing import Dict, List, Tuple, Optional

# Configuración para suprimir los warnings de pdfminer
logging.getLogger('pdfminer').setLevel(logging.ERROR)

# Bibliotecas de terceros
import pdfplumber
import pandas as pd
from tqdm.notebook import tqdm  # Para mostrar barras de progreso

# Si estás usando Google Colab
try:
    from google.colab import files
    from google.colab import drive  # Para acceder a Google Drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 2. CONFIGURACIÓN - Datos externalizados para facilitar mantenimiento
# Constantes y configuraciones
CONFIG = {
    'roles': [
        "Director", "Director Suplente", "Empleador", "Empleado", "Cónyuge",
        "Vicepresidente", "ex", "ex Director", "Presidente", "Firmante cheques",
        "Firmante", "Suplente", "Principal", "Consultado", "Socio"
    ],
    'tipos_empresa': ["SA", "S.A.", "SRL", "S.R.L.", "S.A", "SRL"],
    'prefijos_validos': ["Ex", "ex"],
    'patrones': {
        'cuit': r'^(\d{2}-\d{8,}-\d)',
        'edad': r'\b(\d{1,3}\s+años|fallecido)\b',
        'fecha': r'(\d{2}/\d{2}/\d{4})',
        'arbol_grafo': [r'ÁRBOL(.*?)GRAFO', r'ARBOL(.*?)GRAFO'],
        'espacios': r'\s{2,}',
        'primer_digito': r'(\d)'  # Patrón para encontrar el primer dígito
    },
    'columnas_df': ['CUIT', 'Nombre', 'Rol', 'Edad', 'Fecha', 'Consultado', 'Jerarquia', 'PosicionX', 'Archivo']
}

# 3. DEFINICIÓN DE FUNCIONES

def extraer_texto_pdf(ruta_pdf: str) -> tuple:
    """
    Extrae el texto de un archivo PDF junto con las coordenadas exactas de cada carácter.
    Retorna una tupla con el texto completo y un diccionario de caracteres con coordenadas.
    """
    texto_completo = []
    caracteres_por_linea = {}
    todas_coordenadas = []
    lineas_ordenadas = []  # Lista para mantener las líneas en orden
    arbol_iniciado = False  # Variable para controlar si estamos en la sección ÁRBOL
    arbol_lineas = []       # Lista para almacenar líneas de la sección ÁRBOL

    try:
        # Suprimimos los warnings específicamente durante la extracción
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)

            with pdfplumber.open(ruta_pdf) as pdf:
                print(f"  📄 Información del PDF: {os.path.basename(ruta_pdf)}")
                print(f"  - Número de páginas: {len(pdf.pages)}")

                # Mostramos información de la primera página para diagnóstico
                if len(pdf.pages) > 0:
                    page = pdf.pages[0]
                    print(f"  - MediaBox: {page.mediabox}")
                    print(f"  - CropBox: {page.cropbox or 'No definido (usando MediaBox)'}")

                # Procesamos cada página
                for page_num, pagina in enumerate(pdf.pages):
                    # Extraemos el texto normal para mantener la compatibilidad
                    texto = pagina.extract_text()
                    if texto:
                        texto_completo.append(texto)

                    # Extraemos todos los caracteres con sus coordenadas exactas
                    chars = pagina.chars

                    # Agrupamos caracteres por línea basándonos en la coordenada Y
                    lineas_chars = {}
                    for char in chars:
                        # Redondeamos la coordenada Y para agrupar caracteres en la misma línea
                        y_key = round(char['top'])
                        if y_key not in lineas_chars:
                            lineas_chars[y_key] = []
                        lineas_chars[y_key].append(char)

                    # Ordenamos las líneas por coordenada Y (de arriba hacia abajo)
                    y_keys_ordenadas = sorted(lineas_chars.keys())

                    # Procesamos cada línea
                    for y_key in y_keys_ordenadas:
                        chars_linea = lineas_chars[y_key]
                        # Ordenamos los caracteres dentro de cada línea por coordenada X
                        chars_ordenados = sorted(chars_linea, key=lambda c: c['x0'])

                        # Construimos el texto de la línea
                        texto_linea = ''.join(c['text'] for c in chars_ordenados)

                        # Solo procesamos líneas no vacías
                        if texto_linea.strip():
                            # Guardamos cada carácter con sus coordenadas
                            caracteres_por_linea[texto_linea] = chars_ordenados

                            # Agregamos la línea a la lista ordenada
                            lineas_ordenadas.append(texto_linea)

                            # Detectamos si estamos en la sección ÁRBOL
                            if "ÁRBOL" in texto_linea or "ARBOL" in texto_linea:
                                arbol_iniciado = True
                                print(f"  Encontrada sección ÁRBOL: {texto_linea[:50]}...")
                            elif "GRAFO" in texto_linea and arbol_iniciado:
                                arbol_iniciado = False
                                print(f"  Encontrada sección GRAFO: {texto_linea[:50]}...")
                            elif arbol_iniciado:
                                # Calculamos la posición X del primer carácter no espacio
                                pos_x = None
                                primer_char_no_espacio = None
                                for char in chars_ordenados:
                                    if char['text'].strip():  # Si no es un espacio
                                        pos_x = char['x0']
                                        primer_char_no_espacio = char
                                        break

                                if pos_x is not None:
                                    # Guardamos información detallada para cada línea del árbol
                                    arbol_lineas.append({
                                        'texto': texto_linea,
                                        'posicion_x': pos_x,
                                        'indice': len(lineas_ordenadas) - 1,
                                        'chars': chars_ordenados,
                                        'primer_char': primer_char_no_espacio,
                                        'cuit': extraer_cuit(texto_linea)  # Extraer CUIT
                                    })

                            # También agregamos a la lista global de coordenadas
                            for char in chars_ordenados:
                                todas_coordenadas.append({
                                    'texto': char['text'],
                                    'x0': char['x0'],
                                    'y0': char['top'],
                                    'x1': char['x1'],
                                    'y1': char['bottom'],
                                    'page': page_num,
                                    'linea': texto_linea
                                })

        # Ordenamos las coordenadas globales por página y posición Y
        todas_coordenadas = sorted(todas_coordenadas, key=lambda c: (c['page'], c['y0']))

        # Detectar y corregir valores atípicos en posiciones X
        # Usamos análisis estadístico para identificar y corregir posiciones anómalas
        if len(arbol_lineas) > 5:  # Solo si hay suficientes líneas para analizar
            # Extraer todas las posiciones X
            todas_pos_x = [l['posicion_x'] for l in arbol_lineas]

            # Calcular estadísticas básicas
            media_x = sum(todas_pos_x) / len(todas_pos_x)
            desviacion_x = (sum((x - media_x) ** 2 for x in todas_pos_x) / len(todas_pos_x)) ** 0.5

            # Identificar valores atípicos usando el método de desviación estándar
            for linea in arbol_lineas:
                pos_x = linea['posicion_x']
                # Si la posición está muy lejos de la media (más de 2 desviaciones estándar)
                if abs(pos_x - media_x) > 2 * desviacion_x:
                    # Buscar líneas con CUIT similar para determinar nivel jerárquico correcto
                    cuit_prefix = linea.get('cuit', '').split('-')[0] if linea.get('cuit', '') else ''
                    lineas_similares = [l for l in arbol_lineas
                                       if l.get('cuit', '').startswith(cuit_prefix)
                                       and abs(l['posicion_x'] - pos_x) <= desviacion_x]

                    if lineas_similares:
                        # Usar la mediana de posiciones X de líneas similares
                        pos_similares = sorted([l['posicion_x'] for l in lineas_similares])
                        mediana_x = pos_similares[len(pos_similares) // 2]
                        linea['posicion_x_corregida'] = mediana_x

        # Normalizar posiciones X detectando y agrupando niveles similares
        if arbol_lineas:
            posiciones_x = [l['posicion_x'] for l in arbol_lineas]

            # Detectar grupos de posiciones X similares (niveles jerárquicos)
            grupos_x = agrupar_posiciones_x(posiciones_x)

            # Normalizar las posiciones X basadas en los grupos detectados
            for linea in arbol_lineas:
                for grupo_idx, (min_x, max_x) in enumerate(grupos_x):
                    if min_x <= linea['posicion_x'] <= max_x:
                        # Asignar el valor promedio del grupo como posición normalizada
                        linea['posicion_x_normalizada'] = (min_x + max_x) / 2
                        linea['nivel_jerarquico'] = grupo_idx
                        break

        # Imprimimos diagnóstico de las líneas del árbol
        print("\n📊 Diagnóstico de líneas del árbol:")
        for i, linea in enumerate(arbol_lineas[:min(15, len(arbol_lineas))]):  # Primeras 15 líneas del árbol
            pos_x = linea.get('posicion_x_normalizada', linea['posicion_x'])
            print(f"  {i+1}. X={pos_x:.2f} - {linea['texto'][:50]}...")

        # Analizamos si la indentación es consistente con la estructura jerárquica
        if len(arbol_lineas) > 0:
            # Calculamos la distribución de posiciones X
            posiciones_x = [l.get('posicion_x_normalizada', l['posicion_x']) for l in arbol_lineas]
            posiciones_unicas = sorted(set(posiciones_x))

            print("\n📏 Posiciones X únicas encontradas (indentaciones):")
            for i, pos in enumerate(posiciones_unicas[:10]):  # Mostramos hasta 10 posiciones
                print(f"  Indentación {i+1}: X={pos:.2f} - {posiciones_x.count(pos)} líneas")

        return '\n'.join(texto_completo), caracteres_por_linea, todas_coordenadas, lineas_ordenadas, arbol_lineas
    except Exception as e:
        print(f"❌ Error al extraer texto del PDF {os.path.basename(ruta_pdf)}: {e}")
        import traceback
        traceback.print_exc()
        return "", {}, [], [], []

# Función para extraer CUIT de una línea
def extraer_cuit(linea: str) -> str:
    """Extrae el CUIT de una línea si existe."""
    patron_cuit = re.compile(r'(\d{2}-\d{8,}-\d)')
    m_cuit = patron_cuit.search(linea)
    if m_cuit:
        return m_cuit.group(1)
    return ""

# Función para agrupar posiciones X similares
def agrupar_posiciones_x(posiciones_x: list, umbral: float = 5.0) -> list:
    """
    Agrupa posiciones X similares para identificar niveles jerárquicos utilizando
    un algoritmo adaptativo que no requiere conocimiento previo del número de niveles.
    Retorna una lista de tuplas (min_x, max_x) para cada grupo.
    """
    if not posiciones_x or len(posiciones_x) < 3:
        # Si hay muy pocos puntos, agrupación simple
        if posiciones_x:
            return [(min(posiciones_x), max(posiciones_x))]
        return []

    # Implementación de clusterización jerárquica adaptativa
    # Primero ordenamos las posiciones
    pos_ordenadas = sorted(posiciones_x)

    # Calculamos las diferencias entre puntos consecutivos
    diferencias = [pos_ordenadas[i+1] - pos_ordenadas[i]
                  for i in range(len(pos_ordenadas)-1)]

    # La mediana de las diferencias nos da una idea del espaciado típico entre puntos
    # del mismo grupo
    if diferencias:
        diferencias_ordenadas = sorted(diferencias)
        mediana_diferencia = diferencias_ordenadas[len(diferencias_ordenadas) // 2]

        # Usamos un múltiplo de la mediana como umbral para separar grupos
        # Este enfoque se adapta automáticamente a los datos
        umbral_adaptativo = max(umbral, 2.5 * mediana_diferencia)
    else:
        umbral_adaptativo = umbral

    # Ahora formamos los grupos usando el umbral adaptativo
    grupos = []
    grupo_actual = [pos_ordenadas[0]]

    for pos in pos_ordenadas[1:]:
        # Si la posición está cerca del último valor del grupo actual, agregarla
        if pos - grupo_actual[-1] <= umbral_adaptativo:
            grupo_actual.append(pos)
        else:
            # Si no, cerrar el grupo actual y comenzar uno nuevo
            grupos.append((min(grupo_actual), max(grupo_actual)))
            grupo_actual = [pos]

    # No olvidar el último grupo
    if grupo_actual:
        grupos.append((min(grupo_actual), max(grupo_actual)))

    return grupos

def extraer_seccion(texto_completo: str, patrones: List[str]) -> Optional[str]:
    """Extrae una sección específica del texto según patrones."""
    for patron in patrones:
        coincidencia = re.search(patron, texto_completo, re.DOTALL)
        if coincidencia:
            prefijo = "ÁRBOL" if "ÁRBOL" in patron else "ARBOL"
            return prefijo + coincidencia.group(1)
    return None

def limpiar_texto(texto: str) -> str:
    """Limpia y normaliza el texto extraído."""
    if not texto:
        return ""

    print(f"Limpiando texto, longitud inicial: {len(texto)}")

    # Patrones para la limpieza
    patron_id = re.compile(r'^(\d{2}-\d{8,}-\d)')
    patron_espacios = re.compile(r'\s{2,}')
    roles = CONFIG['roles']
    tipos_empresa = CONFIG['tipos_empresa']
    prefijos_validos = CONFIG['prefijos_validos']

    lineas_limpias = []
    lineas_originales = texto.split('\n')
    print(f"Total de líneas a procesar: {len(lineas_originales)}")

    for i, linea in enumerate(lineas_originales):
        # Saltar líneas vacías o de sistema
        linea = linea.strip()
        if not linea or linea.startswith("NOSIS.Manager") or "Página" in linea:
            continue

        # Imprimir diagnóstico para algunas líneas
        if i < 5 or i % 10 == 0:
            print(f"  Limpiando línea {i}: {linea[:50]}..." if len(linea) > 50 else f"  Limpiando línea {i}: {linea}")

        # Normalizar espacios múltiples primero
        linea = patron_espacios.sub(' ', linea)

        # Buscar líneas que comienzan con CUIT
        if patron_id.match(linea):
            # Ya tiene formato CUIT al inicio, la conservamos como está
            lineas_limpias.append(linea)
            continue

        # Si no comienza con CUIT pero contiene uno, intentamos reformatearla
        cuit_en_texto = re.search(r'(\d{2}-\d{8,}-\d)', linea)
        if cuit_en_texto:
            # Reorganizar para que el CUIT esté al inicio
            cuit = cuit_en_texto.group(1)
            resto = linea.replace(cuit, "").strip()
            linea_reformateada = f"{cuit} {resto}"
            lineas_limpias.append(linea_reformateada)
            if i < 5:
                print(f"    Línea reformateada: {linea_reformateada[:50]}...")
            continue

        # Si llegamos aquí, no pudimos identificar un CUIT
        if i < 5:
            print(f"    ⚠️ No se identificó CUIT, conservando línea original")
        lineas_limpias.append(linea)

    texto_limpio = '\n'.join(lineas_limpias)
    print(f"Limpieza completada, longitud final: {len(texto_limpio)}")
    return texto_limpio

def extraer_y_limpiar_seccion(ruta_pdf: str) -> Tuple[str, str, dict, list, list, list]:
    """
    Combina la extracción y limpieza en una sola función.
    Retorna una tupla con el texto limpio, un mensaje de estado, un diccionario de caracteres,
    una lista de coordenadas, una lista de líneas ordenadas y una lista de líneas del árbol.
    """
    # Extraer todo el texto del PDF junto con las coordenadas
    texto_completo, caracteres_por_linea, todas_coordenadas, lineas_ordenadas, arbol_lineas = extraer_texto_pdf(ruta_pdf)
    if not texto_completo:
        return "", "No se pudo extraer texto del PDF.", {}, [], [], []

    # Extraer la sección específica
    seccion = extraer_seccion(texto_completo, CONFIG['patrones']['arbol_grafo'])
    if not seccion:
        return "", "No se encontraron las secciones ARBOL y GRAFO en el documento.", {}, [], [], []

    # Limpiar el texto de la sección
    texto_limpio = limpiar_texto(seccion)

    return texto_limpio, "Extracción exitosa", caracteres_por_linea, todas_coordenadas, lineas_ordenadas, arbol_lineas

def analizar_linea(linea: str, caracteres_por_linea: dict, todas_coordenadas: list, arbol_lineas: list) -> Optional[Dict]:
    """
    Analiza una línea de texto y extrae sus componentes estructurados.
    Retorna un diccionario con los datos o None si no se pudo analizar.
    """
    # Diagnóstico detallado para cada línea
    print(f"Analizando línea: {linea[:50]}..." if len(linea) > 50 else f"Analizando línea: {linea}")

    linea_original = linea  # Guardamos la línea original para buscar posiciones
    linea = linea.strip()
    if not linea:
        print("  Línea vacía, saltando")
        return None

    # Compilar patrones
    patron_cuit = re.compile(r'^(\d{2}-\d{8,}-\d)')
    patron_edad = re.compile(r'(\d{1,3}\s+años|fallecido)', re.IGNORECASE)
    patron_fecha = re.compile(r'(\d{2}/\d{2}/\d{4})')

    # Extraer CUIT
    m_cuit = patron_cuit.match(linea)
    if not m_cuit:
        print("  No se encontró patrón CUIT en la línea")
        return None

    cuit = m_cuit.group(1)
    print(f"  CUIT encontrado: {cuit}")
    resto = linea[len(cuit):].strip()

    # Buscar el primer dígito en el CUIT para la jerarquía
    jerarquia = ""
    for char in cuit:
        if char.isdigit():
            jerarquia = char
            break

    # Determinar posición X desde las líneas analizadas
    posicion_x = None
    for linea_arbol in arbol_lineas:
        if cuit in linea_arbol['texto']:
            # Prioridad: posición corregida, normalizada, o posición original
            if 'posicion_x_corregida' in linea_arbol:
                posicion_x = linea_arbol['posicion_x_corregida']
            elif 'posicion_x_normalizada' in linea_arbol:
                posicion_x = linea_arbol['posicion_x_normalizada']
            else:
                posicion_x = linea_arbol['posicion_x']
            print(f"  Posición X encontrada: {posicion_x}")
            break

    if posicion_x is None:
        posicion_x = -1
        print("  No se encontró posición X, usando valor por defecto -1")

    # Inicializar datos
    datos = {
        'CUIT': cuit,
        'Nombre': "",
        'Rol': "",
        'Edad': "",
        'Fecha': "",
        'Consultado': "No",
        'Jerarquia': jerarquia,
        'PosicionX': posicion_x
    }

    # Nueva estrategia: Usar expresiones regulares más flexibles para extraer partes
    # Este enfoque será más tolerante a variaciones en el formato

    # 1. Intentar encontrar roles
    roles_pattern = '|'.join(CONFIG['roles'])
    patron_rol = re.compile(f"({roles_pattern})", re.IGNORECASE)
    m_rol = patron_rol.search(resto)

    if m_rol:
        rol = m_rol.group(1)
        pos_rol = m_rol.start()
        print(f"  Rol encontrado: {rol} en posición {pos_rol}")

        # Extraer nombre (todo lo que está antes del rol)
        nombre = resto[:pos_rol].strip()
        datos['Nombre'] = nombre
        datos['Rol'] = rol

        # Resto después del rol
        resto_despues = resto[pos_rol + len(rol):].strip()

        # Extraer edad
        m_edad = patron_edad.search(resto_despues)
        if m_edad:
            datos['Edad'] = m_edad.group(1).strip()
            if datos['Edad'].lower() == "fallecido":
                datos['Edad'] = "Fallecido"
            print(f"  Edad encontrada: {datos['Edad']}")

        # Extraer fecha
        m_fecha = patron_fecha.search(resto_despues)
        if m_fecha:
            datos['Fecha'] = m_fecha.group(1)
            print(f"  Fecha encontrada: {datos['Fecha']}")

        # Verificar si está consultado
        datos['Consultado'] = "Sí" if "Consultado" in resto_despues else "No"

        print(f"  Registro procesado exitosamente: {datos}")
        return datos
    else:
        print("  No se encontró un rol válido en la línea")

    # Si llegamos aquí, no se pudo analizar la línea correctamente
    return None

def convertir_a_dataframe(texto_limpio: str, caracteres_por_linea: dict, todas_coordenadas: list, arbol_lineas: list) -> Tuple[pd.DataFrame, int]:
    """
    Convierte el texto limpio en un DataFrame estructurado.
    Retorna el DataFrame y el número de líneas problemáticas.
    """
    print(f"Convirtiendo texto a DataFrame, longitud texto: {len(texto_limpio)}")
    if not texto_limpio:
        print("⚠️ Texto limpio vacío, no se puede crear DataFrame")
        return pd.DataFrame(columns=CONFIG['columnas_df'][:-1]), 0  # Excluimos 'Archivo'

    datos = []
    lineas_problema = 0
    lineas_procesadas = 0

    for i, linea in enumerate(texto_limpio.split('\n')):
        if not linea.strip():
            continue

        lineas_procesadas += 1
        print(f"\nProcesando línea {lineas_procesadas}: {linea[:50]}..." if len(linea) > 50 else f"\nProcesando línea {lineas_procesadas}: {linea}")

        # Si encontramos NOSIS.Manager o líneas de sistema, las ignoramos
        if "NOSIS.Manager" in linea or "Página" in linea:
            print("  Ignorando línea de sistema")
            continue

        registro = analizar_linea(linea, caracteres_por_linea, todas_coordenadas, arbol_lineas)
        if registro:
            datos.append(list(registro.values()))
            print(f"  ✅ Línea convertida a registro: {registro}")
        else:
            lineas_problema += 1
            print(f"  ❌ No se pudo analizar la línea")

    print(f"\nResumen de conversión: {len(datos)} registros creados, {lineas_problema} líneas problemáticas")
    if datos:
        df = pd.DataFrame(datos, columns=CONFIG['columnas_df'][:-1])  # Excluimos 'Archivo'
        print(f"DataFrame creado con {len(df)} filas y {len(df.columns)} columnas")
        return df, lineas_problema
    else:
        print("⚠️ No se crearon registros, DataFrame vacío")
        return pd.DataFrame(columns=CONFIG['columnas_df'][:-1]), lineas_problema

def mostrar_estadisticas(df: pd.DataFrame) -> None:
    """Muestra estadísticas básicas del DataFrame."""
    if df.empty:
        print("❌ No se extrajeron datos estructurados.")
        return

    print(f"📊 Total registros: {len(df)}")
    print(f"🔎 Roles únicos: {df['Rol'].nunique()}")
    print(f"Distribución de roles:")
    print(df['Rol'].value_counts().head())
    print(f"📊 Distribución de jerarquías:")
    print(df['Jerarquia'].value_counts())
    print(f"📏 Rango de posiciones X: {df['PosicionX'].min()} - {df['PosicionX'].max()}")
    print("\n🧪 Primeros registros:")
    print(df.head().to_string())

def procesar_archivo(ruta_pdf: str) -> pd.DataFrame:
    """
    Procesa un único archivo PDF y retorna su DataFrame.
    """
    print(f"\n📂 Procesando archivo: {ruta_pdf}")

    try:
        # Extraer y limpiar texto
        texto_extraido, mensaje, caracteres_por_linea, todas_coordenadas, lineas_ordenadas, arbol_lineas = extraer_y_limpiar_seccion(ruta_pdf)
        if not texto_extraido:
            print(f"⚠️ {mensaje}")
            return pd.DataFrame(columns=CONFIG['columnas_df'])

        print(f"📝 Texto extraído correctamente")

        # Vamos a mostrar las primeras líneas del texto extraído para diagnóstico
        print("\nMuestra del texto extraído:")
        lineas_muestra = texto_extraido.split('\n')[:10]  # Primeras 10 líneas
        for i, linea in enumerate(lineas_muestra):
            print(f"  {i+1}. {linea[:70]}..." if len(linea) > 70 else f"  {i+1}. {linea}")

        # Información sobre las líneas del árbol encontradas
        print(f"\n📋 Se encontraron {len(arbol_lineas)} líneas en la sección ÁRBOL")

        # Mostrar algunas líneas del árbol para diagnóstico
        print("\n📊 Muestra de líneas del árbol con sus posiciones X:")
        for i, linea_info in enumerate(arbol_lineas[:10]):  # Primeras 10 líneas
            print(f"  {i+1}. X={linea_info['posicion_x']:.2f} - {linea_info['texto'][:50]}...")

        # Convertir a DataFrame
        df, lineas_problema = convertir_a_dataframe(texto_extraido, caracteres_por_linea, todas_coordenadas, arbol_lineas)
        if lineas_problema > 0:
            print(f"⚠️ No se pudieron analizar {lineas_problema} líneas.")

        if df.empty:
            print("⚠️ No se extrajeron datos estructurados.")
            return pd.DataFrame(columns=CONFIG['columnas_df'])

        # Agregar columna con el nombre del archivo
        df['Archivo'] = os.path.basename(ruta_pdf)

        # Mostrar estadísticas
        mostrar_estadisticas(df)

        return df
    except Exception as e:
        print(f"❌ Error procesando {ruta_pdf}: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame(columns=CONFIG['columnas_df'])

def procesar_archivos_batch(archivos: List[str], max_workers: int = 1) -> pd.DataFrame:
    """
    Procesa múltiples archivos en paralelo y combina sus resultados.
    Reducido a 1 worker para mejor diagnóstico.
    """
    if not archivos:
        print("No se proporcionaron archivos para procesar.")
        return pd.DataFrame(columns=CONFIG['columnas_df'])

    todos_dfs = []

    # Procesamiento secuencial para mejor diagnóstico
    if max_workers == 1:
        for archivo in tqdm(archivos, desc="Procesando archivos"):
            df = procesar_archivo(archivo)
            if not df.empty:
                todos_dfs.append(df)
    else:
        # Procesamiento paralelo para múltiples archivos
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            resultados = list(tqdm(
                executor.map(procesar_archivo, archivos),
                total=len(archivos),
                desc="Procesando archivos"
            ))

        for df in resultados:
            if not df.empty:
                todos_dfs.append(df)

    # Combinar resultados
    if todos_dfs:
        resultado_final = pd.concat(todos_dfs, ignore_index=True)
        print(f"\n📚 Archivos combinados: {len(todos_dfs)} con un total de {len(resultado_final)} registros.")
        return resultado_final
    else:
        print("\n⚠️ No se obtuvieron datos de ningún archivo.")
        return pd.DataFrame(columns=CONFIG['columnas_df'])

def cargar_desde_drive(ruta_drive: str = "Desarrollo/insumo_prueba") -> List[str]:
    """
    Carga archivos PDF desde una carpeta específica de Google Drive.

    Args:
        ruta_drive: Ruta relativa a la raíz de Google Drive

    Returns:
        Lista de rutas a archivos PDF encontrados
    """
    # Verificar si estamos en Colab
    if not IN_COLAB:
        print("Esta función solo está disponible en Google Colab.")
        return []

    try:
        # Montar Google Drive
        drive.mount('/content/drive')

        # Construir la ruta completa - Nota: "Mi unidad" se mapea a "/content/drive/MyDrive" en Colab
        ruta_completa = f"/content/drive/MyDrive/{ruta_drive}"

        # Verificar que la carpeta existe
        if not os.path.exists(ruta_completa):
            print(f"⚠️ La carpeta {ruta_completa} no existe.")
            # Intentemos listar las carpetas en el directorio raíz para ayudar al usuario
            print("\nCarpetas disponibles en la raíz de Google Drive:")
            root_dir = "/content/drive/MyDrive"
            if os.path.exists(root_dir):
                print([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
            return []

        print(f"📂 Buscando archivos PDF en: {ruta_completa}")

        # Listar archivos PDF en la carpeta
        pdf_files = [
            os.path.join(ruta_completa, f) for f in os.listdir(ruta_completa)
            if f.lower().endswith('.pdf')
        ]

        if not pdf_files:
            print("⚠️ No se encontraron archivos PDF en la carpeta especificada.")
        else:
            print(f"✅ Se encontraron {len(pdf_files)} archivos PDF.")
            for pdf in pdf_files:
                print(f"  - {os.path.basename(pdf)}")

        return pdf_files

    except Exception as e:
        print(f"❌ Error al acceder a Google Drive: {e}")
        # Mostrar un mensaje más útil
        print("\nVerifica que:")
        print("1. Tu Google Drive está correctamente montado")
        print("2. La ruta es correcta (normalmente comienza desde 'MyDrive' no desde 'Mi unidad')")
        print("3. Tienes permisos para acceder a esa carpeta")
        return []

def main() -> pd.DataFrame:
    """
    Función principal que coordina el proceso de extracción.
    Solicita archivos al usuario y los procesa.
    """
    print("🔍 Iniciando extractor de PDF a DataFrame")

    # Opciones para cargar archivos
    if IN_COLAB:
        print("\nSelecciona cómo quieres cargar los archivos:")
        print("1. Buscar PDFs en el directorio actual")
        print("2. Usar archivos desde Google Drive (Desarrollo/insumo_prueba)")
        print("3. Especificar otra carpeta en Google Drive")
        print("4. Subir archivos PDF manualmente")

        opcion = input("\nIngresa el número de la opción (1-4): ").strip()

        if opcion == "1":
            # Buscar en el directorio actual
            current_dir = os.getcwd()
            pdfs_encontrados = [f for f in os.listdir(current_dir) if f.lower().endswith('.pdf')]
            if pdfs_encontrados:
                print(f"Se encontraron {len(pdfs_encontrados)} PDFs en el directorio actual.")
                rutas_archivos = [os.path.join(current_dir, pdf) for pdf in pdfs_encontrados]
            else:
                print("No se encontraron PDFs en el directorio actual.")
                return pd.DataFrame(columns=CONFIG['columnas_df'])
        elif opcion == "2":
            # Usar la ruta predeterminada
            rutas_archivos = cargar_desde_drive()
        elif opcion == "3":
            # Pedir una ruta personalizada
            print("\nNota: En Colab, 'Mi unidad' se accede como 'MyDrive'")
            ruta_personalizada = input("\nIngresa la ruta dentro de Google Drive (ej: Carpeta/Subcarpeta): ")
            rutas_archivos = cargar_desde_drive(ruta_personalizada)
        elif opcion == "4":
            # Subir archivos manualmente
            print("\nPor favor, sube tus archivos PDF:")
            uploaded = files.upload()
            rutas_archivos = list(uploaded.keys())
        else:
            print("⚠️ Opción no válida. Buscando PDFs en el directorio actual.")
            current_dir = os.getcwd()
            pdfs_encontrados = [f for f in os.listdir(current_dir) if f.lower().endswith('.pdf')]
            if pdfs_encontrados:
                rutas_archivos = [os.path.join(current_dir, pdf) for pdf in pdfs_encontrados]
            else:
                print("No se encontraron PDFs en el directorio actual.")
                return pd.DataFrame(columns=CONFIG['columnas_df'])
    else:
        # Si no estamos en Colab, buscar PDFs en el directorio actual
        current_dir = os.getcwd()
        pdfs_encontrados = [f for f in os.listdir(current_dir) if f.lower().endswith('.pdf')]

        if pdfs_encontrados:
            print(f"Se encontraron {len(pdfs_encontrados)} PDFs en el directorio actual.")
            rutas_archivos = [os.path.join(current_dir, pdf) for pdf in pdfs_encontrados]
        else:
            # Pedir ruta de archivo manualmente
            ruta = input("No se encontraron PDFs. Ingresa la ruta al directorio o archivo PDF: ")
            if os.path.isdir(ruta):
                rutas_archivos = [
                    os.path.join(ruta, f) for f in os.listdir(ruta)
                    if f.lower().endswith('.pdf')
                ]
            else:
                rutas_archivos = [ruta]

    if not rutas_archivos:
        print("No se encontraron archivos PDF para procesar.")
        return pd.DataFrame(columns=CONFIG['columnas_df'])

    # Procesar archivos (usando 1 worker para mejor diagnóstico)
    df_final = procesar_archivos_batch(rutas_archivos, max_workers=1)

    # Mostrar el DataFrame final
    print("\n📊 DataFrame Final:")
    if df_final.empty:
        print("No se extrajeron datos de ningún archivo.")
    else:
        # Mostrar estadísticas generales
        print(f"Total de registros: {len(df_final)}")
        print(f"Columnas: {', '.join(df_final.columns)}")
        print(f"Archivos procesados: {df_final['Archivo'].nunique()}")
        print(f"Roles encontrados: {df_final['Rol'].nunique()}")
        print(f"Jerarquías encontradas: {df_final['Jerarquia'].nunique()}")
        print(f"Rango de posiciones X: {df_final['PosicionX'].min()} - {df_final['PosicionX'].max()}")

        # Mostrar una muestra del DataFrame
        print("\nMuestra del DataFrame:")
        pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
        print(df_final.head(10).to_string())

        # Buscar específicamente la fila con el CUIT problemático
        if '23-05061924-9' in df_final['CUIT'].values:
            print("\nFila con CUIT 23-05061924-9 (MEDICA JORGE LUIS):")
            print(df_final[df_final['CUIT'] == '23-05061924-9'].to_string())

        # Guardar en CSV
        try:
            csv_path = 'resultados_estructurados.csv'
            df_final.to_csv(csv_path, index=False)
            print(f"\n✅ Resultados guardados en: {csv_path}")
        except Exception as e:
            print(f"\n❌ Error al guardar CSV: {e}")

    return df_final

# Ejecutar el script si se ejecuta directamente
if __name__ == "__main__":
    try:
        df_final = main()

        # Configurar pandas para mostrar todos los datos
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_rows', 100)
        pd.set_option('display.width', 1000)

        # Mostrar resultados finales
        print("\n\n📊 RESULTADOS FINALES DEL PROCESAMIENTO:")
        if df_final.empty:
            print("⚠️ No se obtuvieron resultados.")
        else:
            # Verificar posición X para el CUIT problemático
            if '23-05061924-9' in df_final['CUIT'].values:
                fila_problema = df_final[df_final['CUIT'] == '23-05061924-9']
                print(f"\n✅ Se procesó correctamente el CUIT problemático 23-05061924-9:")
                print(f"   Posición X: {fila_problema['PosicionX'].values[0]}")
                print(f"   Nombre: {fila_problema['Nombre'].values[0]}")
                print(f"   Rol: {fila_problema['Rol'].values[0]}")

            # Mostrar estadísticas de posiciones X por nivel jerárquico
            print("\n📊 Estadísticas por jerarquía:")
            for jerarquia in sorted(df_final['Jerarquia'].unique()):
                if not pd.isna(jerarquia) and jerarquia:  # Ignorar valores vacíos o NaN
                    grupo = df_final[df_final['Jerarquia'] == jerarquia]
                    if not grupo.empty:
                        print(f"\nJerarquía {jerarquia}:")
                        print(f"  - Registros: {len(grupo)}")
                        print(f"  - PosiciónX media: {grupo['PosicionX'].mean():.2f}")
                        print(f"  - PosiciónX min-max: {grupo['PosicionX'].min():.2f} - {grupo['PosicionX'].max():.2f}")

    except Exception as e:
        print(f"\n❌ Error en la ejecución principal: {e}")
        import traceback
        traceback.print_exc()

🔍 Iniciando extractor de PDF a DataFrame

Selecciona cómo quieres cargar los archivos:
1. Buscar PDFs en el directorio actual
2. Usar archivos desde Google Drive (Desarrollo/insumo_prueba)
3. Especificar otra carpeta en Google Drive
4. Subir archivos PDF manualmente

Ingresa el número de la opción (1-4): 2
Mounted at /content/drive
📂 Buscando archivos PDF en: /content/drive/MyDrive/Desarrollo/insumo_prueba
✅ Se encontraron 4 archivos PDF.
  - 0043 - IF-2023-05785127-APN-DI%OA.pdf
  - 0042 - IF-2023-05782103-APN-DI%OA.pdf
  - 0044 - IF-2023-05789752-APN-DI%OA.pdf
  - 0045 - IF-2023-06337560-APN-DI%OA.pdf


Procesando archivos:   0%|          | 0/4 [00:00<?, ?it/s]


📂 Procesando archivo: /content/drive/MyDrive/Desarrollo/insumo_prueba/0043 - IF-2023-05785127-APN-DI%OA.pdf
  📄 Información del PDF: 0043 - IF-2023-05785127-APN-DI%OA.pdf
  - Número de páginas: 15
  - MediaBox: (0, 0, 595, 842)
  - CropBox: (0, 0, 595, 842)
  Encontrada sección ÁRBOL: ÁRBOL...
  Encontrada sección GRAFO: GRAFO...

📊 Diagnóstico de líneas del árbol:
  1. X=30.21 - 30-66647634-0 HILCOR IMPLEMENTOS AGRICOLAS SAPrinc...
  2. X=42.42 - 23-05061924-9 MEDICA JORGE LUISPresidente75 años19...
  3. X=56.65 - 30-70813233-7 NUEVAS TIERRAS SAFirmante cheques18 ...
  4. X=56.65 - 30-56132500-2 YOMEL SAEmpleador44 años01/05/2017...
  5. X=56.65 - 30-66647634-0 HILCOR IMPLEMENTOS AGRICOLAS SAEmple...
  6. X=20.00 - NOSIS.Manager | Informe Individual | Cli.64008 | 0...
  7. X=56.65 - 27-05317972-5 ALONSO SILVIA SUSANACónyuge74 años02...
  8. X=56.65 - 30-56132500-2 YOMEL SAPresidente44 años08/03/2016...
  9. X=42.42 - 23-23756494-4 MEDICA MARIA SILVINADirector47 años1...
  10. X=56.65

In [ ]:
df_final

,CUIT,Nombre,Rol,Edad,Fecha,Consultado,Jerarquia,PosicionX,Archivo
0,30-66647634-0,HILCOR IMPLEMENTOS AGRICOLAS SA,Principal,26 años,,Sí,3,20.000000,0043 - IF-2023-05785127-APN-DI%OA.pdf
1,23-05061924-9,MEDICA JORGE LUIS,Presidente,75 años,19/09/2016,No,2,42.417015,0043 - IF-2023-05785127-APN-DI%OA.pdf
2,30-70813233-7,NUEVAS TIERRAS SA,Firmante cheques,18 años,16/08/2019,No,3,56.652862,0043 - IF-2023-05785127-APN-DI%OA.pdf
3,30-56132500-2,YOMEL SA,Empleador,44 años,01/05/2017,No,3,56.652862,0043 - IF-2023-05785127-APN-DI%OA.pdf
4,30-66647634-0,HILCOR IMPLEMENTOS AGRICOLAS SA,Empleador,26 años,01/05/2017,Sí,3,20.000000,0043 - IF-2023-05785127-APN-DI%OA.pdf
5,27-05317972-5,ALONSO SILVIA SUSANA,Cónyuge,74 años,02/02/2016,No,2,56.652862,0043 - IF-2023-05785127-APN-DI%OA.pdf
6,30-56132500-2,YOMEL SA,Presidente,44 años,08/03/2016,No,3,56.652862,0043 - IF-2023-05785127-APN-DI%OA.pdf
7,23-23756494-4,MEDICA MARIA SILVINA,Director,47 años,19/09/2016,No,2,42.417015,0043 - IF-2023-05785127-APN-DI%OA.pdf
8,30-56132500-2,YOMEL SA,Empleador,44 años,01/11/2015,No,3,56.652862,0043 - IF-2023-05785127-APN-DI%OA.pdf
9,30-66647634-0,HILCOR IMPLEMENTOS AGRICOLAS SA,Empleador,26 años,01/11/2015,Sí,3,20.000000,0043 - IF-2023-05785127-APN-DI%OA.pdf
